In [2]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Load dataset
column_names = [
    'Class', 'Alcohol', 'Malic_acid', 'Ash', 'Alcalinity_of_ash', 'Magnesium',
    'Total_phenols', 'Flavanoids', 'Nonflavanoid_phenols', 'Proanthocyanins',
    'Color_intensity', 'Hue', 'OD280_OD315_of_diluted_wines', 'Proline'
]
wine_data = pd.read_csv('wine.data', header=None, names=column_names)


In [3]:
# Preprocess data
X = wine_data.drop('Class', axis=1)
y = wine_data['Class']
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.2, random_state=42, stratify=y
)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long) - 1  # PyTorch class starts at 0
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long) - 1


In [4]:
# Define MLP Model
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activation_fn):
        super(SimpleMLP, self).__init__()
        layers = []
        current_size = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(current_size, hidden_size))
            layers.append(activation_fn())
            current_size = hidden_size
        layers.append(nn.Linear(current_size, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


In [5]:
# Training and evaluation function
def train_and_evaluate(hidden_sizes, activation_fn, learning_rate, epochs, batch_size):
    def create_dataloader(X, y, batch_size):
        dataset = TensorDataset(X, y)
        return DataLoader(dataset, batch_size=batch_size, shuffle=True)

    train_loader = create_dataloader(X_train_tensor, y_train_tensor, batch_size)
    test_loader = create_dataloader(X_test_tensor, y_test_tensor, batch_size)

    # Model initialization
    model = SimpleMLP(X_train.shape[1], hidden_sizes, len(y.unique()), activation_fn)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    accuracy = correct / total
    return accuracy


In [6]:
# Hyperparameter combinations
hidden_layers_options = [[4], [8, 8], [16, 16, 16]]
activation_functions = [nn.ReLU, nn.Sigmoid, nn.Tanh, nn.Identity]  # Identity for linear
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]
epochs_options = [1, 10, 25, 50, 100, 250]
batch_sizes = [16, 32, 64, 128, 256, 512]

# Run experiments
results = []
for hidden_layers in hidden_layers_options:
    for activation_fn in activation_functions:
        for lr in learning_rates:
            for epochs in epochs_options:
                for batch_size in batch_sizes:
                    accuracy = train_and_evaluate(hidden_layers, activation_fn, lr, epochs, batch_size)
                    results.append({
                        'Hidden Layers': hidden_layers,
                        'Activation Function': activation_fn.__name__,
                        'Learning Rate': lr,
                        'Epochs': epochs,
                        'Batch Size': batch_size,
                        'Accuracy': accuracy
                    })

# Save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('mlp_classification_results.csv', index=False)
print("Results saved to 'mlp_classification_results.csv'")


Results saved to 'mlp_classification_results.csv'
